In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")

In [4]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(36980, 674)
(36980,)
(20984, 674)
(20984,)


In [5]:
X_train

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,5,0,0,0,0,0,7,0,0,0,...,0,0,18,0,0,0,0,0,2023-01-29 00:00:00,HB101
1,24,0,0,0,0,0,7,0,0,0,...,0,0,3,0,0,0,0,0,2023-01-29 12:00:00,HB101
2,2,0,0,0,0,0,13,0,0,0,...,0,0,13,0,0,0,0,0,2023-01-30 00:00:00,HB101
3,34,0,0,0,0,0,18,0,0,0,...,0,0,17,0,0,0,0,0,2023-01-30 12:00:00,HB101
4,1,0,0,0,0,0,11,0,0,0,...,0,0,24,0,0,0,0,0,2023-01-31 00:00:00,HB101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36975,36,0,0,0,0,0,24,0,0,0,...,0,0,1,0,0,0,0,0,2023-08-29 12:00:00,JC116
36976,1,0,0,0,0,0,10,0,0,0,...,0,0,27,0,0,0,0,0,2023-08-30 00:00:00,JC116
36977,32,0,0,0,0,0,22,0,0,0,...,0,0,5,0,0,0,0,0,2023-08-30 12:00:00,JC116
36978,3,0,0,0,0,0,13,0,0,0,...,0,0,24,0,0,0,0,0,2023-08-31 00:00:00,JC116


In [6]:
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()

In [7]:
pipeline

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x00000213B8B98360>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [8]:
# Get your pipeline
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()

# Train
pipeline.fit(X_train, y_train)

# Predict
y_pred = pipeline.predict(X_test)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015662 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10690
[LightGBM] [Info] Number of data points in the train set: 36980, number of used features: 115
[LightGBM] [Info] Start training from score 6.905057


In [9]:
y_pred

array([ 0.69948779, 44.2803079 ,  5.29287255, ..., 15.19298491,
        2.03246091, 11.08187414])

In [10]:
from sklearn.metrics import mean_absolute_error

In [11]:
test_mae = mean_absolute_error(y_test, y_pred)
print(f"{test_mae:.4f}")

2.6619


In [12]:

import src.config as config

In [13]:
import joblib  


# Save the pipeline  
joblib.dump(pipeline, config.MODELS_DIR / "lgb_model.pkl")

['C:\\Users\\2003j\\OneDrive\\Desktop\\citibike_ride_prediction-main\\models\\lgb_model.pkl']

In [14]:
import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

2025-05-11 01:25:53,868 INFO: Initializing external client
2025-05-11 01:25:53,870 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 01:25:54,611 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215645


In [15]:
# Create a feature view if it doesn't already exist
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all(),
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) created successfully.")
except Exception as e:
    print(f"Error creating feature view: {e}")

# Retrieve the feature view
try:
    feature_view = feature_store.get_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) retrieved successfully.")
except Exception as e:
    print(f"Error retrieving feature view: {e}")

Error creating feature view: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1215645/featurestores/1203276/featureview). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270179,"usrMsg":"Feature view: time_series_hourly_feature_view_citi_bike, version: 1","errorMsg":"The provided feature view name and version already exists"}', error code: 270179, error msg: The provided feature view name and version already exists, user msg: Feature view: time_series_hourly_feature_view_citi_bike, version: 1
Feature view 'time_series_hourly_feature_view_citi_bike' (version 1) retrieved successfully.


In [16]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [17]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name=config.MODEL_NAME,
    metrics={"test_mae": test_mae},
    description="LightGBM model for bike rides",
    input_example=X_train.sample(),
    model_schema=model_schema,
    #include_files=["src/", "requirements.txt"]  # Include directories and files
)
# https://community.hopsworks.ai/t/attributeerror-windowspath-object-has-no-attribute-startswith/1003
model.save(str(config.MODELS_DIR) + "/lgb_model.pkl")
# model.save('C:\\Users\\mkhaw\\github\\sp25_taxi\\models\\lgb_model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading C:\Users\2003j\OneDrive\Desktop\citibike_ride_prediction-main\models/lgb_model.pkl: 0.000%|         …

Uploading c:\Users\2003j\OneDrive\Desktop\citibike_ride_prediction-main\notebooks\input_example.json: 0.000%| …

Uploading c:\Users\2003j\OneDrive\Desktop\citibike_ride_prediction-main\notebooks\model_schema.json: 0.000%|  …

Model created, explore it at https://c.app.hopsworks.ai:443/p/1215645/models/model_demand_predictor_next6hours/12


Model(name: 'model_demand_predictor_next6hours', version: 12)

In [18]:
model

Model(name: 'model_demand_predictor_next6hours', version: 12)

In [19]:
models = model_registry.get_models(name=config.MODEL_NAME)

In [20]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-11 01:26:11,822 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 01:26:11,829 INFO: Initializing external client
2025-05-11 01:26:11,830 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 01:26:12,612 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215645


Downloading: 0.000%|          | 0/309539 elapsed<00:00 remaining<?

In [21]:
model

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x00000213B8B98360>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])